In [2]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import torch
import os
import tqdm
import json
import shutil
%matplotlib inline

In [38]:
class ObjectDetector:
    def __init__(self, model_path=None):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f'device = {self.device}')
        if model_path is None:
            self.model = YOLO('yolov8m.pt').to(self.device)
        else:
            self.model = YOLO(model_path).to(self.device)
            
    def predict_img(self, image_path, conf=0.5):
        if not os.path.isfile(image_path):
            raise Exception('Path is not a file')
        return self.model(source=image_path, conf=conf, stream=False)
    
    def predict_dir(self, dir_path, conf=0.5):
        if not os.path.isdir(dir_path):
            raise Exception('Path not found or is not a directory')
        return self.model(source=dir_path, conf=conf, stream=True)
    @staticmethod
    def get_image_metadata(result, cnt, dataset_id=1):
        image_metadata = {}
        height, width = result.orig_shape
        image_metadata['id'] = cnt
        image_metadata['dataset_id'] = dataset_id
        image_metadata['path'] = result.path
        image_metadata['height'], image_metadata['width'] = height, width
        image_metadata['file_name'] = os.path.basename(result.path)
        return image_metadata

    def add_coco(self, coco_output, image_metadata, result, annotation_id):
        coco_output["images"].append({
            "id": image_metadata["id"],
            "dataset_id": image_metadata["dataset_id"],
            "category_ids": [],
            "path": os.path.join('datasets', image_metadata["path"]),
            "width": image_metadata["width"],
            "height": image_metadata["height"],
            "file_name": image_metadata["file_name"],
            "annotated": False,
            "annotating": [],
            "num_annotations": 0,
            "metadata": {},
            "deleted": False,
            "milliseconds": 0,
            "events": [],
            "regenerate_thumbnail": False
        })
        for x in result.boxes:
            xmin, ymin, xmax, ymax = x.cpu().numpy().xyxy[0]
            xmin, ymin, xmax, ymax = xmin.item(), ymin.item(), xmax.item(), ymax.item()
            ymin, xmin, ymax, xmax = round(ymin, 1), round(xmin, 1), round(ymax, 1), round(xmax, 1)
            x, y, w, h = xmin, ymin, (xmax - xmin), (ymax - ymin)
            x, y, w, h = round(x, 1), round(y, 1), round(w, 1), round(h, 1)
            segmentation_points = [xmin, ymin, xmax, ymin, xmax, ymax, xmin, ymax]
            coco_output["annotations"].append({
                "id": annotation_id,
                "image_id": image_metadata["id"],
                "category_id": 1,
                "segmentation": [segmentation_points], 
                "area": round(w * h, 0),
                "bbox": [x, y, w, h],
                "iscrowd": False,
                "isbbox": True,
                "color": "#8eb517", 
                "metadata": {}
            })
            annotation_id += 1
        return annotation_id

    def remove_empty(self, dir_path, split_path='SLONI_empty'):
        os.makedirs(split_path, exist_ok=True)
        for filename in tqdm.tqdm(os.listdir(dir_path)):
            file_path = os.path.join(dir_path, filename)
            results = self.predict_img(file_path, conf=0.4)
            if len(result.boxes) == 0:
                shutil.move(file_path, os.path.join(split_path, filename))
            
        
    def coco_annotate(self, dir_path, output_path='labels.json'):
        results = self.predict_dir(dir_path)
        coco_output = {
            "images": [],
            "categories": [
                {"id": 1, "name": "Elephant", "supercategory": "", "color": "#3ab7dd", "metadata": {}, "keypoint_colors": []}
            ],
            "annotations": []
        }
        annotation_id = 1
        for i, result in enumerate(results, 1):
            image_metadata = self.get_image_metadata(result, i)
            annotation_id = self.add_coco(coco_output, image_metadata, result, annotation_id)
        print(coco_output)
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(coco_output, f, ensure_ascii=False, indent=4)
        
    def show(self, img):
        plt.figure()
        plt.axis('off')
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) 
        plt.show()
    def show_labeled(self, img_path):
        result = self.predict_image(img_path)
        labeled_img = result[0].plot()
        self.show(labeled_img)

        
        
object_detector = ObjectDetector('best.pt')
# object_detector.coco_annotate('cx')
# object_detector.coco_annotate('../SLONI_label/')
object_detector.coco_annotate('../SLONI_label')

device = cuda

image 1/3688 /home/improcloud/nguyehu7/BP/../SLONI_label/screenshot1_13_02__11_30.png: 352x640 1 Elephant, 11.6ms
image 2/3688 /home/improcloud/nguyehu7/BP/../SLONI_label/screenshot1_20_02__14_20.png: 352x640 2 Elephants, 11.0ms
image 3/3688 /home/improcloud/nguyehu7/BP/../SLONI_label/screenshot1_20_02__14_25.png: 352x640 2 Elephants, 11.0ms
image 4/3688 /home/improcloud/nguyehu7/BP/../SLONI_label/screenshot1_20_02__14_30.png: 352x640 3 Elephants, 11.1ms
image 5/3688 /home/improcloud/nguyehu7/BP/../SLONI_label/screenshot1_20_02__14_35.png: 352x640 3 Elephants, 11.0ms
image 6/3688 /home/improcloud/nguyehu7/BP/../SLONI_label/screenshot1_20_02__14_40.png: 352x640 3 Elephants, 15.8ms
image 7/3688 /home/improcloud/nguyehu7/BP/../SLONI_label/screenshot1_20_02__14_45.png: 352x640 2 Elephants, 11.1ms
image 8/3688 /home/improcloud/nguyehu7/BP/../SLONI_label/screenshot1_20_02__14_50.png: 352x640 2 Elephants, 11.6ms
image 9/3688 /home/improcloud/nguyehu7/BP/../SLONI_label/screensho

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [33]:
# import random
# def move(dir_path, split_path='SLONI_rand'):
#     os.makedirs(split_path, exist_ok=True)
#     for filename in tqdm.tqdm(os.listdir(dir_path)):
#         file_path = os.path.join(dir_path, filename)
#         if random.random() < 0.4 and filename[0:11] in ['screenshot2', 'screenshot7']:
#             shutil.move(file_path, os.path.join(split_path, filename))
#         if random.random() < 0.3 and filename[0:11] in ['screenshot1']:
#             shutil.move(file_path, os.path.join(split_path, filename))
# move('../SLONI_label')

100%|███████████████████████████████████| 5361/5361 [00:00<00:00, 101469.14it/s]


In [35]:
mp = {}
dir_path = '../SLONI_label'
for filename in tqdm.tqdm(os.listdir(dir_path)):
    name = filename[0:11]
    if name not in mp:
        mp[name] = 1
    else:
        mp[name] += 1
print(mp)

100%|██████████████████████████████████| 3688/3688 [00:00<00:00, 1783534.32it/s]

{'screenshot2': 1047, 'screenshot7': 963, 'screenshot4': 355, 'screenshot1': 827, 'screenshot6': 496}
